In [1]:
#libraries:

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import urllib.request
import pandas as pd

In [2]:
#Initialising Lists
all_links = []
all_pages = []

In [3]:
%%time
#Scraping Pages
link = 'https://pk.sapphireonline.pk/collections/kids'
all_pages.append(link)
base_link = 'https://pk.sapphireonline.pk'
while (True):
        uClient = uReq(link)
        all_web = uClient.read()
        uClient.close()
        page_soup = soup(all_web, "html.parser")
        pages = page_soup.find('div', {'class':'infinite-scrolling'})
        if (pages == None):
            break
        link = base_link + pages.a['href']
        all_pages.append(link)

all_pages
        

Wall time: 4.65 s


['https://pk.sapphireonline.pk/collections/kids',
 'https://pk.sapphireonline.pk/collections/kids?page=2']

In [4]:
%%time
#Scraping All Product Links
product_container = []
for link in all_pages:
    uClient = uReq(link)
    all_web = uClient.read()
    uClient.close()
    page_soup = soup(all_web, "html.parser")
    for product in page_soup.findAll("div", {"class":"grid-item col-6 col-md-4 col-xl-3 four-columns"}):
        product_container.append(product)
    #Retrieving All Product Links
    for link in range(len(product_container)):
        x = base_link + product_container[link].a['href']
        all_links.append(x)

all_rows = []
len(all_links)

Wall time: 3.62 s


118

In [5]:
%%time
#Scraping All Products


h = 0
x = 0
faultylinks = []
for link in all_links:
    h += 1
    if (h%5 == 0):    
        print('working...' + str(x))
    
    try:
        uClient = uReq(link)
        all_web = uClient.read()
        uClient.close()
        name = None
        des1 = None
        des2 = None
        detail = None
        color = None
        fabric = None
        oldprice = None
        newprice = None
        all_sizes = None
        extra_detail = None
        product_soup = soup(all_web, "html.parser")
        name = product_soup.find("h2", {"itemprop":"name"}).text.strip()
        content = product_soup.find("section", {"id":'content1'})('p')
        name = product_soup.find("h2", {"itemprop":"name"}).text.strip()
        if (len(content) != 1 and len(content) <= 5):
            if (content[1].text != 'Ready To Wear'):
                if (content[0].strong != None):
                    des1 = content[0].strong.text
                else:
                    b = content[0].text.find("\n")
                    des1 = content[0].text[:b].strip()

                des2 = content[0].text.replace(des1, "").strip()
                details = content[1].text
                z = details.find("Color")
                detail = details[8:z].strip()
                l = details.find("Fabric")
                color = details[z+6:l].strip()
                fabric = details[l+7:].strip()
                if len(content) == 3:
                    extra_detail = content[2].text
            else:
                des1 = content[2].text.strip()
                des2 = content[0].text.strip()
                detail = content[1].text.strip()
                z = content[3].text.find("Fabric")
                color = content[3].text[:z].replace("Color:", "").strip()
                fabric = content[3].text[z:].replace("Fabric:", "").strip()
        elif (len(content) >= 6):
            des1 = content[3].text.strip()
            des2 = content[1].text.strip()
            detail = content[4].text.strip()
            z = content[5].text.find("Fabric")
            color = content[5].text[:z].replace("Color:", "").strip()
            fabric = content[5].text[z:].replace("Fabric:", "").strip()

        else:
            content = content[0]
            des1 = content.strong.text.strip()
            content = content.text
            z = content.find("Details:")
            l = content.find("Color")
            k = content.find("Fabric")
            j = content.find("SIZE")
            des2 = content[len(des1):z].strip()
            detail = content[z:l].replace("Details:", "").strip()
            color = content[l:k].replace("Color:", "").strip()
            fabric = content[k:j].replace("Fabric:", "").strip()


        prices = product_soup.find("div",{"class":"prices"})('span')
        oldprice = prices[1].text.strip()
        newprice = prices[2].text.strip()
        sizes = product_soup.find_all("div", {'class':'swatch-element'})
        all_sizes = []
        types = []
        for size in sizes:
            c = size.label.text.strip()
            if (c == 'XXS' or c == 'XS' or c == 'S' or c == 'M' or c == 'L' or c == 'XL' or c == 'XXL' or c == range(0,50)):
                all_sizes.append(c)
            else:
                types.append(c)

        all_sizes = ','.join(all_sizes)
        types = ','.join(types)
        #Images
        #images = product_soup.find_all("a", {"class":"fancybox"})
        #y = 0
        #for a in images:
            #file_name = str(x) + "." + str(y) + ".jpg"
            #print(a['href'] + file_name)
           # y += 1
            #urllib.request.urlretrieve("https:" + a['href'], file_name)
        
        x += 1
        row = {'Name':name,'Detail':detail,'Main-Description': des1,'Sub-Description':des2, 'Old-Price': oldprice, 'New-Price': newprice,'Color': color, 'Fabric' : fabric, 'Types': types, 'All Sizes' : all_sizes,"Extra Detail" : extra_detail}
        all_rows.append(row)
    except:
        print(link)
        faultylinks.append(link)
        pass

working...4
working...9
working...14
working...19
working...24
working...29
working...34
working...39
working...44
working...49
working...54
working...59
working...64
working...69
working...74
working...79
working...84
working...89
working...94
working...99
working...104
working...109
working...114
Wall time: 3min 31s


In [6]:
products = pd.DataFrame(all_rows, columns = ['Name','Detail','Main-Description', 'Sub-Description','Old-Price','New-Price','Color', 'Fabric','Types','All Sizes',"Extra Detail"])
products

,Name,Detail,Main-Description,Sub-Description,Old-Price,New-Price,Color,Fabric,Types,All Sizes,Extra Detail
0,Pop Star,"A Line Kurta With Adda On Front Yoke, Boat Nec...",Kids Girls - 2-Piece A line Kurta with Trouser,This heavily embellished look features a lavis...,"Rs.6,090.00","Rs.3,828.50",Pink,Viscose Chiffon Straight Jamawar Trouser Color...,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,None
1,Vanilla,"Detailing On Band Collar, Front Placket And Sl...",Kids Boys - Straight Kurta,This ivory coloured dobby kurta features light...,"Rs.1,790.00","Rs.1,033.50",Cream,Dobby,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,None
2,Foam,White Non-Stretch Trouser,Kids Boys - Non-Stretchable Trouser,These white cambric trousers are perfect to we...,Rs.890.00,Rs.448.50,White,Cambric,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,None
3,Boat,White Stretch Trouser,Kids Boys - Stretchable Trouser,These white trousers have an excellent stretch...,Rs.890.00,Rs.448.50,White,Cotton Lycra,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,None
4,Paper,Plain White Shalwar,Kids Boys - Plain Basic Shalwar,"This classic white shalwar is sturdy, durable ...",Rs.890.00,Rs.448.50,White,Cambric,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,None
5,Whale,"Plain Front with Embroidery, Back Slit with Do...",3-Piece Kaliyan Styled Shirt with Gharara & Du...,This gorgeous three piece look comes in shades...,"Rs.7,590.00","Rs.4,803.50",Off White & Blue,Viscose Masoori,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,None
6,Lilac Grey,"A Line shirt with Adda work on Boat Neck, Plai...",2-Piece A Line Shirt with Trouser,This pink perfection comprises of a zari net j...,"Rs.7,090.00","Rs.4,478.50",Pink,Zari Net Jacquard,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,Straight Pants Color: Pink Fabric: Jamawar
7,Grape,Straight Shirt with Adda work on Neck & Front ...,3-Piece Straight Shirt with Gharara & Dupatta,Adda style work adorns the viscose masoori shi...,"Rs.8,090.00","Rs.5,128.50",Plum,Viscose Masoori,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,None
8,Twinkle,"Embroidered Front Panel with round Neck, Plain...",2-Piece A-Line Shirt with Trouser,Boot cut raw silk trousers add a fancy touch t...,"Rs.5,090.00","Rs.3,178.50",Cream Pearl,Viscose Zarri Net,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,Boot Cut Pants Color: Cream Pearl Fabric: Visc...
9,King,"Straight Front, Plain Back, Plain Sleeves",Kids Boys - Sherwani,This solid black sherwani is crafted from luxe...,"Rs.5,090.00","Rs.3,178.50",Black,Jamawar,"3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,12Y",,None


In [7]:
products.to_csv('Sapphire-Kids.csv', encoding='utf-8')